In [2]:
import numpy as np
from scapy.all import sniff
from scapy.layers.inet import IP
import time


# Número máximo de paquetes a capturar
num_paquetes = 10

# Listas para almacenar las características de los paquetes
times = []
src_ips = []
dst_ips = []
protocols = []
lengths = []

# Función para manejar cada paquete capturado
def packet_handler(packet):
    global times, src_ips, dst_ips, protocols, lengths
    if IP in packet:
        times.append(time.time())  # Registro de tiempo de llegada del paquete
        src_ips.append(packet[IP].src)
        dst_ips.append(packet[IP].dst)
        protocols.append(packet[IP].proto)
        lengths.append(len(packet))

        # Si se han capturado suficientes paquetes, detener la captura
        if len(src_ips) >= num_paquetes:
            return True  # Devuelve True para detener la captura

# Captura los paquetes y llama a packet_handler para cada uno
sniff(prn=packet_handler, store=0, count=num_paquetes)

# Organizar los datos en un numpy array
data = np.vstack((times, src_ips, dst_ips, protocols, lengths)).T

# Imprimir el array
print("Datos capturados:\n", data)


True
Datos capturados:
 [['1712222482.4289496' '157.90.91.73' '192.168.100.2' '6' '66']
 ['1712222483.1362083' '192.168.100.2' '224.0.0.251' '17' '126']
 ['1712222483.1412082' '192.168.100.12' '224.0.0.251' '17' '82']
 ['1712222483.1422062' '192.168.100.12' '224.0.0.251' '17' '126']
 ['1712222483.1432083' '192.168.100.12' '224.0.0.251' '17' '404']
 ['1712222483.1432083' '192.168.100.12' '224.0.0.251' '17' '193']
 ['1712222483.144207' '192.168.100.12' '224.0.0.251' '17' '82']
 ['1712222483.2172782' '192.168.100.12' '224.0.0.251' '17' '256']
 ['1712222483.2172782' '192.168.100.12' '224.0.0.251' '17' '193']
 ['1712222483.346482' '192.168.100.33' '192.168.100.255' '17' '63']]


In [4]:
import pyshark

def capturar_paquetes(num_paquetes):
    packets = []

    def packet_handler(pkt):
        packet_info = {
            "time": pkt.sniff_time.timestamp(),
            "src_ip": pkt.ip.src,
            "dst_ip": pkt.ip.dst,
            "protocol": pkt.transport_layer,
            "length": pkt.length
        }
        packets.append(packet_info)

        if len(packets) >= num_paquetes:
            return False

    capture = pyshark.LiveCapture(interface='your_interface', bpf_filter='tcp')
    capture.apply_on_packets(packet_handler, timeout=10)

    return packets

# Ejemplo de uso
num_paquetes = 25
paquetes = capturar_paquetes(num_paquetes)

# Imprimir información sobre los paquetes capturados
for i, packet in enumerate(paquetes, 1):
    print("Paquete", i)
    print("Tiempo:", packet["time"])
    print("IP origen:", packet["src_ip"])
    print("IP destino:", packet["dst_ip"])
    print("Protocolo:", packet["protocol"])
    print("Longitud:", packet["length"])
    print()


RuntimeError: Cannot run the event loop while another loop is running